In [ ]:
#################################################
# Client Notebook Example
#################################################

# Import the client class
from udp_client import UDPClient

# Create a client with custom settings
client = UDPClient(
    client_id="client_papermill_1",  # Custom ID for papermill execution
    port=37020,
    max_runtime=300,  # Run for 5 minutes
    max_messages=100  # Stop after receiving 100 messages
)

# Define a custom function for client interaction
def client_interaction(client, elapsed_time, event_count):
    """Interact with discovered servers periodically"""
    # Only check every 10 seconds
    if int(elapsed_time) % 10 != 0:
        return True
        
    # Get known servers
    servers = list(client.server_states.keys())
    
    if servers and hasattr(client_interaction, 'last_action_time') and \
       int(elapsed_time) - client_interaction.last_action_time >= 10:
        # Choose a server to interact with
        server_addr = servers[0]
        server_ip, server_port = server_addr
        
        # Randomly choose an action to perform based on elapsed time
        action_selector = int(elapsed_time) % 4
        
        if action_selector == 0:
            # Get server status
            print(f"\nRequesting status from server at {server_ip}:{server_port}")
            
            def status_callback(success, data, error):
                if success:
                    print(f"Server status: {data}")
                else:
                    print(f"Status request failed: {error}")
                    
            client.get_server_status(server_addr, status_callback)
            
        elif action_selector == 1:
            # Set a parameter
            param_name = "client_preference"
            param_value = f"Setting from {client.id} at {int(elapsed_time)}s"
            
            print(f"\nSetting parameter '{param_name}' on server")
            
            def param_callback(success, data, error):
                if success:
                    print(f"Parameter set: {data}")
                else:
                    print(f"Parameter set failed: {error}")
                    
            client.set_server_parameter(server_addr, param_name, param_value, param_callback)
            
        elif action_selector == 2:
            # Send custom command
            action = "echo"
            data = f"Hello from {client.id} at {int(elapsed_time)}s"
            
            print(f"\nSending custom echo command to server")
            
            def echo_callback(success, data, error):
                if success:
                    print(f"Echo response: {data}")
                else:
                    print(f"Echo command failed: {error}")
                    
            client.send_custom_command(server_addr, action, data, echo_callback)
            
        elif action_selector == 3:
            # Request client list
            print(f"\nRequesting client list from server")
            
            def clients_callback(success, data, error):
                if success:
                    client_count = data.get("client_count", 0)
                    clients = data.get("clients", {})
                    print(f"Server has {client_count} connected clients")
                    for addr, info in clients.items():
                        print(f"- {info.get('ip')}:{info.get('port')} (msgs: {info.get('message_count', 0)})")
                else:
                    print(f"Client list request failed: {error}")
                    
            client.get_server_clients(server_addr, clients_callback)
        
        # Update last action time
        client_interaction.last_action_time = int(elapsed_time)
        
    # Initialize last action time if not set
    if not hasattr(client_interaction, 'last_action_time'):
        client_interaction.last_action_time = 0
        
    return True  # Continue execution

print(f"Starting UDP client with ID: {client.id}")
print("Press Ctrl+C to stop")

# Run the client with the interaction function
client_results = client.run(between_events_func=client_interaction)

# Results are now available in client_results for papermill to access
print(f"Client execution complete. Received {len(client.received_messages)} messages.")